In [2]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# Define thruster strengths (random example values)
T_f, T_b, T_l, T_r, T_t, T_d = 2,3,4,1,7,3

# Unit vectors for each face (assuming initial alignment with axes)
vectors = {
    'Front': np.array([T_f, 0, 0]),
    'Back': np.array([-T_b, 0, 0]),
    'Left': np.array([0, T_l, 0]),
    'Right': np.array([0, -T_r, 0]),
    'Top': np.array([0, 0, T_t]),
    'Bottom': np.array([0, 0, -T_d])
}

# get side with most thrust
side_with_most_thrust = max(vectors, key=lambda k: np.linalg.norm(vectors[k]))
side_with_most_thrust_index = list(vectors.keys()).index(side_with_most_thrust)

# get sides adjacent to side with most thrust
adjacent_sides_map = {
    'Front': ['Left', 'Right', 'Top', 'Bottom'],
    'Back': ['Left', 'Right', 'Top', 'Bottom'],
    'Left': ['Front', 'Back', 'Top', 'Bottom'],
    'Right': ['Front', 'Back', 'Top', 'Bottom'],
    'Top': ['Front', 'Back', 'Left', 'Right'],
    'Bottom': ['Front', 'Back', 'Left', 'Right']
}

adjacent_sides = adjacent_sides_map[side_with_most_thrust]

# get adjacent side with most thrust
side_with_most_thrust_adjacent = max(adjacent_sides, key=lambda k: np.linalg.norm(vectors[k]))
side_with_most_thrust_adjacent_index = list(vectors.keys()).index(side_with_most_thrust_adjacent)

# get the adjacent sides that are NOT the side with the most thrust, the side with the second most thrust, and the side opposite of that with the most thrust
sides_to_avoid = [side_with_most_thrust, side_with_most_thrust_adjacent]
sides_to_avoid_indices = [list(vectors.keys()).index(side) for side in sides_to_avoid]

# get the side opposite of the side with the most thrust
opposite_side_map = {
    'Front': 'Back',
    'Back': 'Front',
    'Left': 'Right',
    'Right': 'Left',
    'Top': 'Bottom',
    'Bottom': 'Top'
}

opposite_side = opposite_side_map[side_with_most_thrust]
opposite_side_index = list(vectors.keys()).index(opposite_side)

second_opposite_side = opposite_side_map[side_with_most_thrust_adjacent]
second_opposite_side_index = list(vectors.keys()).index(second_opposite_side)

sides_to_avoid_indices.append(opposite_side_index)
sides_to_avoid_indices.append(second_opposite_side_index)

# find remaining adjacent sides
remaining_sides_indices = [i for i in range(6) if i not in sides_to_avoid_indices]
# get the index of the remaining adjacent side with the most thrust
side_with_most_thrust_remaining_index = max(remaining_sides_indices, key=lambda k: np.linalg.norm(vectors[list(vectors.keys())[k]]))

print (f"side_with_most_thrust_index: {side_with_most_thrust_index}")
print (f"side_with_most_thrust_adjacent_index: {side_with_most_thrust_adjacent_index}")
print (f"side_with_most_thrust_remaining_index: {side_with_most_thrust_remaining_index}")


print(f"Side with most thrust: {side_with_most_thrust}")
angles = np.arange(0, 360, 1)
max_magnitude = 0
best_config = None

# Consider rotations about all three axes in sequence
for angle_x in angles:
    for angle_y in angles:
        for angle_z in angles:

            # Create a composite rotation
            rotation = R.from_euler('xyz', [angle_x, angle_y, angle_z], degrees=True)

            # Heuristic: only consider configurations where the most thrust, second most thrust, and third most thrust thrusters are on
            thrusters_on = [False] * 6
            thrusters_on[side_with_most_thrust_index] = True
            thrusters_on[side_with_most_thrust_adjacent_index] = True
            thrusters_on[side_with_most_thrust_remaining_index] = True

            resultant_vector = np.zeros(3)
            for i, (key, vec) in enumerate(vectors.items()):
                rotated_vec = rotation.apply(vec) if thrusters_on[i] else np.zeros(3)
                resultant_vector += rotated_vec

            magnitude = np.linalg.norm(resultant_vector)

            if magnitude > max_magnitude:
                max_magnitude = magnitude
                best_config = (angle_x, angle_y, angle_z, thrusters_on, resultant_vector)

# Print the best configuration found
angle_x, angle_y, angle_z, thrusters_on, best_vector = best_config

print(f"Angles (degrees): {angle_x}, {angle_y}, {angle_z}")
print(f"Thrusters on: {thrusters_on}")
print(f"Resultant force vector: {best_vector} with magnitude {max_magnitude}")


side_with_most_thrust_index: 4
side_with_most_thrust_adjacent_index: 2
side_with_most_thrust_remaining_index: 1
Side with most thrust: Top
Angles (degrees): 11, 21, 279
Thrusters on: [False, True, True, False, True, False]
Resultant force vector: [2.54882123 0.46923648 8.20264149] with magnitude 8.602325267042634
